In [1]:
library(tidyverse)
library(dplyr)
library(readxl)
library(ggpubr)
library(lme4)
library(ggplot2)
library(broom)
library(purrr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::recode() masks car::recode()
✖ purrr::some()   masks car::some()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


필요한 패키지를 로딩중입니다: Matrix




다음의 패키지를 부착합니다: ‘Matrix’




The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




# Preliminary Data Analysis

## 1. Load Data

In [2]:
patient_df <- read.delim("/Users/jennylee/Downloads/MGH_Olink_COVID_Apr_27_2021/MGH_COVID_Clinical_Info.txt", header=TRUE, sep=";")
head(patient_df)
print(paste0("This dataframe has ", nrow(patient_df), " rows in total."))
glimpse(patient_df)

,subject_id,COVID,Age_cat,BMI_cat,HEART,LUNG,KIDNEY,DIABETES,HTN,IMMUNO,⋯,crp_3_cat,ddimer_3_cat,ldh_3_cat,abs_neut_7_cat,abs_lymph_7_cat,abs_mono_7_cat,creat_7_cat,crp_7_cat,ddimer_7_cat,ldh_7_cat
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1,1,4,0,0,0,0,0,0,⋯,1,1,1,NA,NA,NA,NA,NA,NA,NA
2,2,1,2,2,0,0,0,0,0,0,⋯,2,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,3,1,3,4,0,1,0,0,0,0,⋯,3,2,3,NA,NA,NA,NA,NA,NA,NA
4,4,1,1,2,0,0,0,0,0,0,⋯,2,2,3,NA,NA,NA,NA,NA,NA,NA
5,5,1,3,3,0,0,0,1,1,0,⋯,5,3,NA,3,5,3,1,4,4,3
6,6,1,1,1,1,0,0,0,0,0,⋯,4,3,5,3,3,2,1,4,5,5


[1] "This dataframe has 384 rows in total."


Rows: 384
Columns: 44
$ subject_id      <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,…
$ COVID           <int> 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, …
$ Age_cat         <int> 1, 2, 3, 1, 3, 1, 2, 4, 5, 2, 3, 2, 4, 4, 4, 3, 3, 2, …
$ BMI_cat         <int> 4, 2, 4, 2, 3, 1, 2, 3, 2, 2, 2, 4, 4, 3, 2, 3, 1, 2, …
$ HEART           <int> 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, …
$ LUNG            <int> 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, …
$ KIDNEY          <int> 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, …
$ DIABETES        <int> 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, …
$ HTN             <int> 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, …
$ IMMUNO          <int> 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, …
$ Resp_Symp       <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, …
$ Fever_Sympt     <int> 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, …
$ GI_Symp         

In [3]:
patient_df <- read.delim("/Users/jennylee/Downloads/MGH_Olink_COVID_Apr_27_2021/MGH_COVID_OLINK_NPX.txt", header=TRUE, sep=";")
head(patient_df)
print(paste0("This dataframe has ", nrow(patient_df), " rows in total."))
glimpse(patient_df)

,SampleID,subject_id,Timepoint,Index,OlinkID,UniProt,Assay,MissingFreq,Panel,Panel_Lot_Nr,PlateID,QC_Warning,Assay_Warning,Normalization,LOD,NPX
,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,1_D0,1,D0,56,OID21311,Q9BTE6,AARSD1,0.0000,Oncology,B04404,20200772_Plate5,PASS,PASS,Intensity,-1.0810,1.9024
2,1_D0,1,D0,56,OID20921,Q96IU4,ABHD14B,0.0000,Neurology,B04406,20200772_Plate5,PASS,PASS,Intensity,-1.4574,-0.2625
3,1_D0,1,D0,56,OID21280,P00519,ABL1,0.0013,Oncology,B04404,20200772_Plate5,PASS,PASS,Intensity,-2.4697,0.2659
4,1_D0,1,D0,56,OID21269,P09110,ACAA1,0.1248,Oncology,B04404,20200772_Plate5,PASS,PASS,Intensity,-0.2027,0.5311
5,1_D0,1,D0,56,OID20159,P16112,ACAN,0.0000,Cardiometabolic,B04405,20200772_Plate5,PASS,PASS,Intensity,-3.3481,-2.0366
6,1_D0,1,D0,56,OID20105,Q9BYF1,ACE2,0.0561,Cardiometabolic,B04405,20200772_Plate5,PASS,PASS,Intensity,-1.9055,-1.4261


[1] "This dataframe has 1271808 rows in total."


Rows: 1,271,808
Columns: 16
$ SampleID      <chr> "1_D0", "1_D0", "1_D0", "1_D0", "1_D0", "1_D0", "1_D0", …
$ subject_id    <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ Timepoint     <chr> "D0", "D0", "D0", "D0", "D0", "D0", "D0", "D0", "D0", "D…
$ Index         <int> 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, …
$ OlinkID       <chr> "OID21311", "OID20921", "OID21280", "OID21269", "OID2015…
$ UniProt       <chr> "Q9BTE6", "Q96IU4", "P00519", "P09110", "P16112", "Q9BYF…
$ Assay         <chr> "AARSD1", "ABHD14B", "ABL1", "ACAA1", "ACAN", "ACE2", "A…
$ MissingFreq   <dbl> 0.0000, 0.0000, 0.0013, 0.1248, 0.0000, 0.0561, 0.8127, …
$ Panel         <chr> "Oncology", "Neurology", "Oncology", "Oncology", "Cardio…
$ Panel_Lot_Nr  <chr> "B04404", "B04406", "B04404", "B04404", "B04405", "B0440…
$ PlateID       <chr> "20200772_Plate5", "20200772_Plate5", "20200772_Plate5",…
$ QC_Warning    <chr> "PASS", "PASS", "PASS", "PASS", "PASS", "PASS", "PASS", …
$ Assay_Warn